In [1]:
import numpy as np
import pandas as pd
import ast
import re
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [2]:
df=pd.read_csv("/kaggle/input/transactions/synthetic_fraud_data.csv",nrows=5000)

In [3]:
df

,transaction_id,customer_id,card_number,timestamp,merchant_category,merchant_type,merchant,amount,currency,country,...,device,channel,device_fingerprint,ip_address,distance_from_home,high_risk_merchant,transaction_hour,weekend_transaction,velocity_last_hour,is_fraud
0,TX_a0ad2a2a,CUST_72886,6646734767813109,2024-09-30 00:00:01.034820+00:00,Restaurant,fast_food,Taco Bell,294.87,GBP,UK,...,iOS App,mobile,e8e6160445c935fd0001501e4cbac8bc,197.153.60.199,0,False,0,False,"{'num_transactions': 1197, 'total_amount': 334...",False
1,TX_3599c101,CUST_70474,376800864692727,2024-09-30 00:00:01.764464+00:00,Entertainment,gaming,Steam,3368.97,BRL,Brazil,...,Edge,web,a73043a57091e775af37f252b3a32af9,208.123.221.203,1,True,0,False,"{'num_transactions': 509, 'total_amount': 2011...",True
2,TX_a9461c6d,CUST_10715,5251909460951913,2024-09-30 00:00:02.273762+00:00,Grocery,physical,Whole Foods,102582.38,JPY,Japan,...,Firefox,web,218864e94ceaa41577d216b149722261,10.194.159.204,0,False,0,False,"{'num_transactions': 332, 'total_amount': 3916...",False
3,TX_7be21fc4,CUST_16193,376079286931183,2024-09-30 00:00:02.297466+00:00,Gas,major,Exxon,630.60,AUD,Australia,...,iOS App,mobile,70423fa3a1e74d01203cf93b51b9631d,17.230.177.225,0,False,0,False,"{'num_transactions': 764, 'total_amount': 2201...",False
4,TX_150f490b,CUST_87572,6172948052178810,2024-09-30 00:00:02.544063+00:00,Healthcare,medical,Medical Center,724949.27,NGN,Nigeria,...,Chrome,web,9880776c7b6038f2af86bd4e18a1b1a4,136.241.219.151,1,False,0,False,"{'num_transactions': 218, 'total_amount': 4827...",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,TX_93151e3b,CUST_83367,6280807665253825,2024-09-30 00:59:42.144714+00:00,Grocery,online,Walmart Grocery,0.16,GBP,UK,...,Firefox,web,a188c564f731450d1d8d5476f6ef1c3c,36.205.92.25,1,False,0,False,"{'num_transactions': 204, 'total_amount': 5551...",True
4996,TX_cdce45f0,CUST_64957,375144760088917,2024-09-30 00:59:44.478153+00:00,Retail,physical,IKEA,34483.22,RUB,Russia,...,Chrome,web,cf82fcb5e72387d0ea9ef256adf2764e,240.96.200.81,0,False,0,False,"{'num_transactions': 2626, 'total_amount': 938...",False
4997,TX_0efbee32,CUST_95103,6844240998445372,2024-09-30 00:59:46.816044+00:00,Gas,local,Truck Stop,78657.33,RUB,Russia,...,iOS App,mobile,d41664d5a14c57929b00ec5fb1d152f5,25.180.227.147,0,False,0,False,"{'num_transactions': 165, 'total_amount': 2207...",False
4998,TX_3eaa8fe7,CUST_35139,371954204647092,2024-09-30 00:59:47.811640+00:00,Grocery,online,Walmart Grocery,1294.77,AUD,Australia,...,Firefox,web,37a27a9030ea0032e3198441eae28f3c,202.203.249.13,0,False,0,False,"{'num_transactions': 454, 'total_amount': 1150...",False


In [4]:
df["timestamp"]=pd.to_datetime(df["timestamp"])
df["Year"]=df["timestamp"].dt.year
df["month"]=df["timestamp"].dt.month
df["day"]=df["timestamp"].dt.day
df["hour"]=df["timestamp"].dt.hour
df["min"]=df["timestamp"].dt.minute
df["sec"]=df["timestamp"].dt.second
df["micsec"]=df["timestamp"].dt.microsecond
df["tz"]=df["timestamp"].dt.tz
# df=df.drop(columns=["timestamp"])

In [5]:
df["velocity_last_hour"]

0       {'num_transactions': 1197, 'total_amount': 334...
1       {'num_transactions': 509, 'total_amount': 2011...
2       {'num_transactions': 332, 'total_amount': 3916...
3       {'num_transactions': 764, 'total_amount': 2201...
4       {'num_transactions': 218, 'total_amount': 4827...
                              ...                        
4995    {'num_transactions': 204, 'total_amount': 5551...
4996    {'num_transactions': 2626, 'total_amount': 938...
4997    {'num_transactions': 165, 'total_amount': 2207...
4998    {'num_transactions': 454, 'total_amount': 1150...
4999    {'num_transactions': 1164, 'total_amount': 447...
Name: velocity_last_hour, Length: 5000, dtype: object

In [6]:
# asti=ast.literal_eval(df["velocity_last_hour"])
# asti=df["velocity_last_hour"].apply(lambda x: pd.DataFrame([x])
df["velocity_last_hour"]=df["velocity_last_hour"].apply(lambda x: ast.literal_eval(x) if isinstance(x,str) else x )
df_expand=df["velocity_last_hour"].apply(pd.Series)
concaty=pd.concat([df,df_expand],axis=1).drop(columns=["velocity_last_hour","timestamp","transaction_id"])
concaty

,customer_id,card_number,merchant_category,merchant_type,merchant,amount,currency,country,city,city_size,...,hour,min,sec,micsec,tz,num_transactions,total_amount,unique_merchants,unique_countries,max_single_amount
0,CUST_72886,6646734767813109,Restaurant,fast_food,Taco Bell,294.87,GBP,UK,Unknown City,medium,...,0,0,1,34820,UTC,1197.0,3.349856e+07,105.0,12.0,1.925481e+06
1,CUST_70474,376800864692727,Entertainment,gaming,Steam,3368.97,BRL,Brazil,Unknown City,medium,...,0,0,1,764464,UTC,509.0,2.011476e+07,100.0,12.0,5.149117e+06
2,CUST_10715,5251909460951913,Grocery,physical,Whole Foods,102582.38,JPY,Japan,Unknown City,medium,...,0,0,2,273762,UTC,332.0,3.916385e+07,97.0,12.0,1.852242e+06
3,CUST_16193,376079286931183,Gas,major,Exxon,630.60,AUD,Australia,Unknown City,medium,...,0,0,2,297466,UTC,764.0,2.201260e+07,105.0,12.0,2.055798e+06
4,CUST_87572,6172948052178810,Healthcare,medical,Medical Center,724949.27,NGN,Nigeria,Unknown City,medium,...,0,0,2,544063,UTC,218.0,4.827636e+06,88.0,12.0,1.157231e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,CUST_83367,6280807665253825,Grocery,online,Walmart Grocery,0.16,GBP,UK,Unknown City,medium,...,0,59,42,144714,UTC,204.0,5.551523e+06,83.0,12.0,8.942270e+05
4996,CUST_64957,375144760088917,Retail,physical,IKEA,34483.22,RUB,Russia,Unknown City,medium,...,0,59,44,478153,UTC,2626.0,9.385562e+07,105.0,12.0,3.501391e+06
4997,CUST_95103,6844240998445372,Gas,local,Truck Stop,78657.33,RUB,Russia,Unknown City,medium,...,0,59,46,816044,UTC,165.0,2.207122e+07,76.0,11.0,4.641011e+06
4998,CUST_35139,371954204647092,Grocery,online,Walmart Grocery,1294.77,AUD,Australia,Unknown City,medium,...,0,59,47,811640,UTC,454.0,1.150472e+07,103.0,12.0,1.124358e+06


In [7]:
concaty["max_single_amount"]=concaty["max_single_amount"].apply(lambda x: re.sub(r"\W+"," ",x) if isinstance(x,str) else x)

In [8]:
concaty


,customer_id,card_number,merchant_category,merchant_type,merchant,amount,currency,country,city,city_size,...,hour,min,sec,micsec,tz,num_transactions,total_amount,unique_merchants,unique_countries,max_single_amount
0,CUST_72886,6646734767813109,Restaurant,fast_food,Taco Bell,294.87,GBP,UK,Unknown City,medium,...,0,0,1,34820,UTC,1197.0,3.349856e+07,105.0,12.0,1.925481e+06
1,CUST_70474,376800864692727,Entertainment,gaming,Steam,3368.97,BRL,Brazil,Unknown City,medium,...,0,0,1,764464,UTC,509.0,2.011476e+07,100.0,12.0,5.149117e+06
2,CUST_10715,5251909460951913,Grocery,physical,Whole Foods,102582.38,JPY,Japan,Unknown City,medium,...,0,0,2,273762,UTC,332.0,3.916385e+07,97.0,12.0,1.852242e+06
3,CUST_16193,376079286931183,Gas,major,Exxon,630.60,AUD,Australia,Unknown City,medium,...,0,0,2,297466,UTC,764.0,2.201260e+07,105.0,12.0,2.055798e+06
4,CUST_87572,6172948052178810,Healthcare,medical,Medical Center,724949.27,NGN,Nigeria,Unknown City,medium,...,0,0,2,544063,UTC,218.0,4.827636e+06,88.0,12.0,1.157231e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,CUST_83367,6280807665253825,Grocery,online,Walmart Grocery,0.16,GBP,UK,Unknown City,medium,...,0,59,42,144714,UTC,204.0,5.551523e+06,83.0,12.0,8.942270e+05
4996,CUST_64957,375144760088917,Retail,physical,IKEA,34483.22,RUB,Russia,Unknown City,medium,...,0,59,44,478153,UTC,2626.0,9.385562e+07,105.0,12.0,3.501391e+06
4997,CUST_95103,6844240998445372,Gas,local,Truck Stop,78657.33,RUB,Russia,Unknown City,medium,...,0,59,46,816044,UTC,165.0,2.207122e+07,76.0,11.0,4.641011e+06
4998,CUST_35139,371954204647092,Grocery,online,Walmart Grocery,1294.77,AUD,Australia,Unknown City,medium,...,0,59,47,811640,UTC,454.0,1.150472e+07,103.0,12.0,1.124358e+06


In [9]:
concaty["customer_id"].nunique()

3063

In [10]:
new_df=pd.get_dummies(concaty).astype(int)
new_df

,card_number,amount,card_present,distance_from_home,high_risk_merchant,transaction_hour,weekend_transaction,is_fraud,Year,month,...,ip_address_99.213.174.113,ip_address_99.220.38.97,ip_address_99.37.52.40,ip_address_99.47.255.42,ip_address_99.5.233.105,ip_address_99.56.35.255,ip_address_99.61.204.63,ip_address_99.72.234.144,ip_address_99.94.22.213,tz_UTC
0,6646734767813109,294,0,0,0,0,0,0,2024,9,...,0,0,0,0,0,0,0,0,0,1
1,376800864692727,3368,0,1,1,0,0,1,2024,9,...,0,0,0,0,0,0,0,0,0,1
2,5251909460951913,102582,0,0,0,0,0,0,2024,9,...,0,0,0,0,0,0,0,0,0,1
3,376079286931183,630,0,0,0,0,0,0,2024,9,...,0,0,0,0,0,0,0,0,0,1
4,6172948052178810,724949,0,1,0,0,0,1,2024,9,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,6280807665253825,0,0,1,0,0,0,1,2024,9,...,0,0,0,0,0,0,0,0,0,1
4996,375144760088917,34483,0,0,0,0,0,0,2024,9,...,0,0,0,0,0,0,0,0,0,1
4997,6844240998445372,78657,0,0,0,0,0,0,2024,9,...,0,0,0,0,0,0,0,0,0,1
4998,371954204647092,1294,0,0,0,0,0,0,2024,9,...,0,0,0,0,0,0,0,0,0,1


In [11]:
X=new_df.drop(columns=["amount"])
Y=new_df["amount"]

In [12]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

In [13]:
random_reg=RandomForestRegressor()
random_reg.fit(X_train,Y_train)
random_reg.score(X_test,Y_test)

0.28521021584680784

# /# QDL

In [14]:
!pip install cirq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.7/532.7 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.3/69.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.5/596.5 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.8/202.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 29.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 38.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB

In [19]:
from keras.layers import Layer,Dense,Flatten
from keras.models import Sequential
import tensorflow as tf
import cirq

In [25]:
def quantum_circuit(data):
    qubit=[cirq.GridQubit(0,i) for i in range(10)]
    circuit=cirq.Circuit()
    for i,value in enumerate(data):
        q=i%len(qubit)
        circuit.append(cirq.rx(value*2*np.pi).on(qubit[q]))
        circuit.append(cirq.H(qubit[q]))
        circuit.append(cirq.ry(value*2*np.pi).on(qubit[q]))
        circuit.append(cirq.X(qubit[q]))
        circuit.append(cirq.rz(value*2*np.pi).on(qubit[q]))
        circuit.append(cirq.Y(qubit[q]))
        circuit.append(cirq.Z(qubit[q]))
        if (i+2)%len(qubit)<len(qubit):
            circuit.append(cirq.CCX(qubit[q],qubit[(i+1)%len(qubit)],qubit[(i+2)%len(qubit)]))
            circuit.append(cirq.CNOT(qubit[q],qubit[(i+2)%len(qubit)]))
        circuit.append(cirq.measure(qubit[q],key=f"qubit_{i}"))
    return circuit,qubit
def run_quantum_circuit(data):
    results=[]
    for inputs_data in data:
        circuit,qubit=quantum_circuit(inputs_data)
        simulator=cirq.Simulator()
        result=simulator.run(circuit)
        measure0=[]
        for i in range(len(inputs_data)):
            measure0.append(result.measurements[f"qubit_{i}"].flatten()[0])
        results.append(measure0)
    return np.array(results,dtype=np.float32)
class Quantumlayer(Layer):
    def __init__(self):
        super(Quantumlayer,self).__init__()
    def call(self,inputs):
        output=tf.numpy_function(run_quantum_circuit,[inputs],tf.float32)
        output.set_shape(inputs.shape)
        return output
model=Sequential([
    Quantumlayer(),
    Dense(1)
])
model.compile(optimizer='adam',loss='mse',metrics=["mae"])
model.fit(X_train.iloc[:4,:4],Y_train[:4],epochs=5)

Epoch 1/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 737ms/step - loss: 327644800.0000 - mae: 11575.8857
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: 327656928.0000 - mae: 11576.0205
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - loss: 327644320.0000 - mae: 11575.6758
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 327651872.0000 - mae: 11575.6445
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 327653152.0000 - mae: 11576.1094
